## PCA

X_train.shape
(54720, 1440)

In [ ]:
from sklearn.decomposition import PCA
import time

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import numpy as np

%matplotlib inline

st = time.time()
pca = PCA(svd_solver="full")

reduced_data = pca.fit_transform(X_train)

print("time:", time.time() - st)
print(reduced_data.shape)
print(pca.components_.shape)
print(pca.explained_variance_ratio_.shape)
print(pca.components_.mean())
print(np.sum(pca.components_ == 0.) / (1440*1440))

## check variance captured

color = 'rgba(217, 217, 217, 0.14)'

traces = []

traces.append(go.Scatter(
    x=np.arange(pca.explained_variance_ratio_.shape[0]),
    y=pca.explained_variance_ratio_,
    mode='lines+markers',
    marker=dict(size=6, line=dict(color=color, width=0.3), opacity=0.8)
))

plot_data = traces
layout = go.Layout(
        title="variance captured",
        xaxis=dict(title="x"),
        yaxis=dict(title="y")
)
fig = go.Figure(data=plot_data, layout=layout)
py.iplot(fig)

In [254]:
centered_data = X_train - np.mean(X_train, axis=0)

print(reduced_data[735])
print(np.dot( pca.components_, centered_data[735]))

# split to normal and anomaly space
st  = time.time()
num_normal_components = 9

P = pca.components_[:num_normal_components].T
mapper = np.dot(P, P.T)
ones = np.eye(1440)

def spe(x):
#     reconst = np.dot((np.eye(1440) - np.dot(P, P.T)), x)
    reconst = np.dot(ones - mapper, x)
    return np.linalg.norm(reconst)**2

errors = [spe(x) for x in centered_data]
    
print("time:", time.time() - st)

[-7.4388285e-05 -4.2760712e-05  7.1610070e-06 ...  2.2117177e-08
  6.8011609e-08  2.7778597e-09]
[-7.4388277e-05 -4.2760603e-05  7.1610102e-06 ...  2.1954264e-08
  6.8276222e-08  2.7530120e-09]
time: 313.49906373023987


In [37]:
# # define threshold

# phi1 = np.sum(pca.explained_variance_[9:])
# phi2 = np.sum(np.array([i**2 for i in pca.explained_variance_[9:]]))
# phi3 = np.sum(np.array([i**3 for i in pca.explained_variance_[9:]]))

# h0 = 1 - 2*phi1*phi3/(3*phi2**2)
# ca = 2.58 # from standard normal distribution, 99.5 percentile value

# delta = phi1*(ca*np.sqrt(2*phi2*h0**2)/phi1 + 1 + phi2*h0*(h0-1)/(phi1**2))**(1/h0)
# print("delta:", delta)

In [ ]:
# plt.figure(figsize=(15, 7))
# plt.plot(errors)
# plt.hlines(delta,xmin=0, xmax=len(errors), colors="red")
# plt.ylim(0, 1e-4)
# plt.show()